读入储存评论信息的excel表

In [582]:
import pandas as pd
import numpy as np
from xml.sax import ContentHandler, parse
import openpyxl
# Reference https://goo.gl/KaOBG3
class ExcelHandler(ContentHandler):
    def __init__(self):
        self.chars = [  ]
        self.cells = [  ]
        self.rows = [  ]
        self.tables = [  ]
    def characters(self, content):
        self.chars.append(content)
    def startElement(self, name, atts):
        if name=="Cell":
            self.chars = [  ]
        elif name=="Row":
            self.cells=[  ]
        elif name=="Table":
            self.rows = [  ]
    def endElement(self, name):
        if name=="Cell":
            self.cells.append(''.join(self.chars))
        elif name=="Row":
            self.rows.append(self.cells)
        elif name=="Table":
            self.tables.append(self.rows)
 
excelHandler = ExcelHandler()
parse(r'E:\OriganalData\szd.xml', excelHandler)  #文件路径
df = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34372 entries, 0 to 34371
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   wb_id          34372 non-null  object
 1   wb_username    34372 non-null  object
 2   wb_userid      34372 non-null  object
 3   wb_content     34372 non-null  object
 4   wb_createtime  34372 non-null  object
 5   wb_forwardnum  34372 non-null  object
 6   wb_commentnum  34372 non-null  object
 7   wb_likenum     34372 non-null  object
 8   wb_url         34372 non-null  object
 9   wb_mood        34372 non-null  object
dtypes: object(10)
memory usage: 2.6+ MB


删除空行

In [583]:
df[df.wb_userid == '']=np.nan
data=df.dropna(how='any')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33089 entries, 0 to 34371
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   wb_id          33089 non-null  object
 1   wb_username    33089 non-null  object
 2   wb_userid      33089 non-null  object
 3   wb_content     33089 non-null  object
 4   wb_createtime  33089 non-null  object
 5   wb_forwardnum  33089 non-null  object
 6   wb_commentnum  33089 non-null  object
 7   wb_likenum     33089 non-null  object
 8   wb_url         33089 non-null  object
 9   wb_mood        33089 non-null  object
dtypes: object(10)
memory usage: 2.8+ MB


按照时间排序

In [584]:
data = data.sort_values(by='wb_createtime')
data.head()
data.tail()

,wb_id,wb_username,wb_userid,wb_content,wb_createtime,wb_forwardnum,wb_commentnum,wb_likenum,wb_url,wb_mood
27114,J9yED4Cy7,天生就很丧丧丧,2640583133,#蓝盈莹好胜心#看了这个分析才能理清楚这个​,2020-07-04 01:50,6,20,142,https://weibo.com/2640583133/J9yED4Cy7?refer_f...,0.682969237111795
27065,J9yGFxYWs,黄章晋ster,1735918992,蚊子在耳边嗡嗡了一个多小时，打开灯忙半天还是没捉着。突然想，人家要我的只是一顿饭，我要它的却...,2020-07-04 01:55,9,25,27,https://weibo.com/1735918992/J9yGFxYWs?refer_f...,0.00111703752162962
31123,J9B2VkKt3,娱乐揭秘蜀黍,5039775130,杜华：那我今天在脏雨绮身上，看到了王ze的风范。华啥时候能把舌头捋直了说话#杜华烫嘴普通话#​,2020-07-04 07:56,161,192,2730,https://weibo.com/5039775130/J9B2VkKt3?refer_f...,0.992331504896706
31394,J9BtifIs8,央视新闻,2656274875,【#29%新冠病毒样本出现D614G变异#】当地时间3日，针对有实验室研究发现新冠病毒D61...,2020-07-04 09:01,1313,1592,41528,https://weibo.com/2656274875/J9BtifIs8?refer_f...,0.999963296224556
33972,J9Db3aQDd,央视新闻,2656274875,#重庆强降雨#【直播！#重庆江津暴雨河水倒灌街道被淹#】昨起，重庆江津区出现强降雨天气，其中...,2020-07-04 13:21,171,360,4793,https://weibo.com/2656274875/J9Db3aQDd?refer_f...,0.9997817384113


利用 pandas 的to_datetime 方法，把 "wb_createtime" 列的字符类型数据解析成 datetime 对象。

In [585]:
ts = data.copy()
ts['wb_createtime'] = pd.to_datetime(ts['wb_createtime'])
ts.set_index(['wb_createtime'],inplace=True)

In [586]:
ts.axes

[DatetimeIndex(['2010-01-07 19:08:00', '2010-01-11 11:08:00',
                '2010-01-13 15:38:00', '2010-01-14 14:48:00',
                '2010-01-24 10:10:00', '2010-01-25 10:18:00',
                '2010-01-26 09:59:00', '2010-02-08 11:11:00',
                '2010-02-10 16:02:00', '2010-02-19 16:32:00',
                ...
                '2020-07-04 01:15:00', '2020-07-04 01:15:00',
                '2020-07-04 01:17:00', '2020-07-04 01:17:00',
                '2020-07-04 01:17:00', '2020-07-04 01:50:00',
                '2020-07-04 01:55:00', '2020-07-04 07:56:00',
                '2020-07-04 09:01:00', '2020-07-04 13:21:00'],
               dtype='datetime64[ns]', name='wb_createtime', length=33089, freq=None),
 Index(['wb_id', 'wb_username', 'wb_userid', 'wb_content', 'wb_forwardnum',
        'wb_commentnum', 'wb_likenum', 'wb_url', 'wb_mood'],
       dtype='object')]

以月为频率进行重采样

In [587]:
ts['count'] = 1
ts.head()
count = ts['count']['20110101':'20191231']
count = count.resample('M').sum()  #以月为频率，对2011——2019的评论数进行重采样
coun = pd.DataFrame(np.random.rand(108, 4), index=pd.date_range('20110131',periods=108, freq='M'), columns=['num', 'log_num', 'log_active_user', 'INDEX'])
coun['num'] = count.values  #评论数
coun['log_num'] = np.log(coun['num']) #评论数的对数
coun.loc['20110131':'20111231', 'log_active_user'] = 12.8317030992143
coun.loc['20120131':'20121231', 'log_active_user'] = 13.2393001743642
coun.loc['20130131':'20131231', 'log_active_user'] = 13.6562013801946
coun.loc['20140131':'20141231', 'log_active_user'] = 14.1008265708957
coun.loc['20150131':'20151231', 'log_active_user'] = 14.5264992391366
coun.loc['20160131':'20161231', 'log_active_user'] = 14.9338750367073
coun.loc['20170131':'20171231', 'log_active_user'] = 15.2585660338899
coun.loc['20180131':'20181231', 'log_active_user'] = 15.4956052311908
coun.loc['20190131':'20191231', 'log_active_user'] = 15.655083445198  #月活跃用户数的对数
coun['INDEX'] = coun['log_num'] / coun['log_active_user'] #舆情热度值
coun

,num,log_num,log_active_user,INDEX
2011-01-31,51,3.931826,12.831703,0.306415
2011-02-28,71,4.262680,12.831703,0.332199
2011-03-31,99,4.595120,12.831703,0.358107
2011-04-30,80,4.382027,12.831703,0.341500
2011-05-31,106,4.663439,12.831703,0.363431
...,...,...,...,...
2019-08-31,270,5.598422,15.655083,0.357610
2019-09-30,224,5.411646,15.655083,0.345680
2019-10-31,220,5.393628,15.655083,0.344529
2019-11-30,205,5.323010,15.655083,0.340018


将降频后的月度数据存入excel

In [588]:
writer = pd.ExcelWriter(r'E:\水环境\论文修改\数据\7月新数据预处理\月度数据.xlsx',engine='openpyxl')
book = openpyxl.load_workbook(writer.path)
writer.book = book
coun.to_excel(excel_writer=writer,sheet_name='水中毒',encoding='utf-8')
writer.close()

将处理好的数据存入excel，格式xlsx

In [589]:
data.to_excel(r'E:\水环境\论文修改\数据\7月新数据预处理\水中毒.xlsx',sheet_name='Sheet01')